In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

import time
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2 
import itertools

import csv
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

from vgg16_2 import VGG_16_2


import argparse
from data_loader_withid_single_label import get_loader
from data_loader_withid_single_label import get_dataset

import math

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import time
import shutil
import os
import torch.nn.functional as F

# Loading dataset

In [2]:
celeba_image_dir = '/home/name/celeba/stargan/data/celeba/images'
attr_path = '/home/name/celeba/stargan/data/celeba/list_attr_celeba.txt'
id_path = '/home/name/celeba/stargan/data/celeba/identity_CelebA.txt'
selected_attrs = ['Black_Hair', 'Male']
celeba_crop_size = 224
image_size = 224 
batch_size = 40
num_workers = 0
mode1 = 'train' #test or train
mode2 = 'test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
torch.set_default_tensor_type('torch.cuda.FloatTensor')


 


In [3]:
celeba_loader_train, data_train = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode1, num_workers)
celeba_loader_test, data_test = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...
Finished preprocessing the CelebA dataset...


In [4]:
print(len(data_train))
print(len(data_test))

182600
19999


# Validate

In [5]:
def validate_single_label(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)


In [6]:
def validate_single_label_stoch(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



In [7]:
def demographic_parity(model):
    model.to(device)
    model.eval()
    smiling_gen0 = 0
    smiling_gen1=0
    gen0 = 0
    gen1= 0 
    correct = 0
    size = 0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        #print(labels.shape)
        
        labels_gender= labels[:,1]#.long().squeeze(1)
        labels = labels[:,0] #.long().squeeze(1)
        preds = model(im) # _
        values, indices = preds.max(-1)
        #print(labels)
        #print(indices)
        correct += indices.eq(labels).sum()

        smiling_gen1 += (indices*labels_gender).sum()
        smiling_gen0 += (indices*(1-labels_gender)).sum()
        gen1 += labels_gender.sum()
        gen0 += (len(labels_gender) - labels_gender.sum())
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels

        
    print(smiling_gen0/gen0, smiling_gen1/gen1)
    print(smiling_gen0/gen0 - smiling_gen1/gen1)

    print(float(correct)/float(size))
    
    
    

In [8]:
def demographic_parity_mask(model, mask):
    model.to(device)
    model.eval()
    smiling_gen0 = 0
    smiling_gen1=0
    gen0 = 0
    gen1= 0 
    correct = 0
    size = 0
    mask = torch.ones(model.intermed.scales().shape)
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        #print(labels.shape)
        im = im*mask
        labels_gender= labels[:,1]#.long().squeeze(1)
        labels = labels[:,0] #.long().squeeze(1)
        preds = model(im) # _
        values, indices = preds.max(-1)
        #print(labels)
        #print(indices)
        correct += indices.eq(labels).sum()

        smiling_gen1 += (indices*labels_gender).sum()
        smiling_gen0 += (indices*(1-labels_gender)).sum()
        gen1 += labels_gender.sum()
        gen0 += (len(labels_gender) - labels_gender.sum())
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        #print(float(correct)/float(size))
        
    print(smiling_gen0/gen0, smiling_gen1/gen1)
    print(smiling_gen0/gen0 - smiling_gen1/gen1)

    print(float(correct)/float(size))
    
    
    

In [9]:
def equal_odds(model):
    model.to(device)
    model.eval()
    smiling_gen0 = 0
    smiling_gen1=0
    gen0 = 0
    gen1= 0 
    
    smiling_gen0_not = 0
    smiling_gen1_not =0
    gen0_not = 0
    gen1_not= 0 
    
    correct = 0
    size = 0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        #print(labels.shape)
        
        labels_gender= labels[:,1]#.long().squeeze(1)
        labels = labels[:,0] #.long().squeeze(1)
        preds = model(im) # _
        values, indices = preds.max(-1)
        #print(labels)
        #print(indices)
        correct += indices.eq(labels).sum()
        smiling_gen1 += ((indices*labels)*labels_gender).sum()
        smiling_gen0 += ((indices*labels)*(1-labels_gender)).sum()
        gen1 += (labels_gender*labels).sum()
        gen0 += ((1 - labels_gender)*labels).sum()
        ##########
        smiling_gen1_not += ((indices*(1-labels))*labels_gender).sum()
        smiling_gen0_not += ((indices*(1-labels))*(1-labels_gender)).sum()
        
        gen1_not += (labels_gender*(1-labels)).sum()
        gen0_not += ((1 - labels_gender)*(1-labels)).sum()
        
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        #print(float(correct)/float(size))
        
    print(gen0, gen1, smiling_gen0, smiling_gen1)
    print(smiling_gen0/gen0, smiling_gen1/gen1)
    print((smiling_gen0/gen0 - smiling_gen1/gen1))
    print("************")
    print(smiling_gen0_not/gen0_not, smiling_gen1_not/gen1_not)
    print((smiling_gen0_not/gen0_not - smiling_gen1_not/gen1_not))    
    
    print(float(correct)/float(size))
    
    
    

In [10]:
def equal_odds_mask(model, mask):
    model.to(device)
    model.eval()
    smiling_gen0 = 0
    smiling_gen1=0
    gen0 = 0
    gen1= 0 
    
    smiling_gen0_not = 0
    smiling_gen1_not =0
    gen0_not = 0
    gen1_not= 0 
    
    correct = 0
    size = 0
    mask = torch.ones(model.intermed.scales().shape)
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        #print(labels.shape)
        im = im*mask
        labels_gender= labels[:,1]#.long().squeeze(1)
        labels = labels[:,0] #.long().squeeze(1)
        preds = model(im) # _
        values, indices = preds.max(-1)
        #print(labels)
        #print(indices)
        correct += indices.eq(labels).sum()
        smiling_gen1 += ((indices*labels)*labels_gender).sum()
        smiling_gen0 += ((indices*labels)*(1-labels_gender)).sum()
        gen1 += (labels_gender*labels).sum()
        gen0 += ((1 - labels_gender)*labels).sum()
        ##########
        smiling_gen1_not += ((indices*(1-labels))*labels_gender).sum()
        smiling_gen0_not += ((indices*(1-labels))*(1-labels_gender)).sum()
        
        gen1_not += (labels_gender*(1-labels)).sum()
        gen0_not += ((1 - labels_gender)*(1-labels)).sum()
        
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        #print(float(correct)/float(size))
        
    print(gen0, gen1, smiling_gen0, smiling_gen1)
    print(smiling_gen0/gen0, smiling_gen1/gen1)
    print((smiling_gen0/gen0 - smiling_gen1/gen1))
    print("************")
    print(smiling_gen0_not/gen0_not, smiling_gen1_not/gen1_not)
    print((smiling_gen0_not/gen0_not - smiling_gen1_not/gen1_not))    
    
    print(float(correct)/float(size))
    
    
    

In [11]:
def equal_odds_syn(model, mask):
    model.to(device)
    model.eval()
    smiling_gen0 = 0
    smiling_gen1=0
    gen0 = 0
    gen1= 0 
    
    smiling_gen0_not = 0
    smiling_gen1_not =0
    gen0_not = 0
    gen1_not= 0 
    
    
    correct = 0
    size = 0
    #mask = torch.ones(model.intermed.scales().shape)
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        #print(labels.shape)
        
        labels_gender= labels[:,1]#.long().squeeze(1)
        labels = labels[:,0] #.long().squeeze(1)
        preds, _= model(im, mask) # _
        values, indices = preds.max(-1)
        #print(labels)
        #print(indices)
        correct += indices.eq(labels).sum()
        smiling_gen1 += ((indices*labels)*labels_gender).sum()
        smiling_gen0 += ((indices*labels)*(1-labels_gender)).sum()
        gen1 += (labels_gender*labels).sum()
        gen0 += ((1 - labels_gender)*labels).sum()
        ##########
        smiling_gen1_not += (((1-indices)*(1-labels))*labels_gender).sum()
        smiling_gen0_not += (((1-indices)*(1-labels))*(1-labels_gender)).sum()
        
        gen1_not += (labels_gender*(1-labels)).sum()
        gen0_not += ((1 - labels_gender)*(1-labels)).sum()
        
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        #print(float(correct)/float(size))
        
    print(gen0, gen1, smiling_gen0, smiling_gen1)
    print(smiling_gen0/gen0, smiling_gen1/gen1)
    print((smiling_gen0/gen0 - smiling_gen1/gen1))
    print("************")
    print(smiling_gen0_not/gen0_not, smiling_gen1_not/gen1_not)
    print((smiling_gen0_not/gen0_not - smiling_gen1_not/gen1_not))    
    
    print(float(correct)/float(size))
    
    
    

In [12]:
def demographic_parity_syn(model, mask):
    model.to(device)
    model.eval()
    smiling_gen0 = 0
    smiling_gen1=0
    smiling_gen0_not = 0
    smiling_gen1_not=0
    gen0 = 0
    gen1= 0 
    correct = 0
    size = 0
    #mask = torch.ones(model.intermed.scales().shape)
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        #print(labels.shape)
        
        labels_gender= labels[:,1]#.long().squeeze(1)
        labels = labels[:,0] #.long().squeeze(1)
        preds , _ = model(im, mask) # _
        values, indices = preds.max(-1)
        #print(labels)
        #print(indices)
        correct += indices.eq(labels).sum()

        smiling_gen1 += (indices*labels_gender).sum()
        smiling_gen0 += (indices*(1-labels_gender)).sum()
        
        smiling_gen1_not += ((1-indices)*labels_gender).sum()
        smiling_gen0_not += ((1-indices)*(1-labels_gender)).sum()
        gen1 += labels_gender.sum()
        gen0 += (len(labels_gender) - labels_gender.sum())
        del im
        del preds
        
        del values
        
        #print(correct)
        size += len(labels)
        del labels
        #print(float(correct)/float(size))
        
    print(smiling_gen0/gen0, smiling_gen1/gen1)
    print((smiling_gen0/gen0 - smiling_gen1/gen1))
    print("***********************")
    print(smiling_gen0_not/gen0, smiling_gen1_not/gen1)
    print((smiling_gen0_not/gen0 - smiling_gen1_not/gen1))
    print(float(correct)/float(size))
    
    
    

# Noisy Network

In [18]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-5) #-inf


        self.normal = torch.distributions.normal.Normal(0,1)
        self.rhos.requires_grad = True
        self.locs.requires_grad = True
        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self, mask):
        epsilon = self.normal.sample(self.rhos.shape)
        return self.locs + self.scales() * epsilon * mask
                                 
                               
                            
    def forward(self, input, mask):
        noise = self.sample_noise(mask)
        return (input)*mask + noise



class vgg_syn(nn.Module):

    def __init__(self, model_features, model_classifier, min_scale,max_scale, given_locs, given_scale):
        super(vgg_syn, self).__init__()
        

        self.intermed = NoisyActivation( given_locs, given_scale, min_scale, max_scale)
        self.model_pt2 =  torch.nn.Sequential(*(list(model_features)))
        self.model_pt3 = model_classifier
        #self.components = components
        for child in itertools.chain(self.model_pt2, self.model_pt3): #self.model_pt2 #(self.model_pt2, 
            for param in child.parameters():
                param.requires_grad = False
            if isinstance(child, nn.modules.batchnorm._BatchNorm):
                child.eval()
                child.affine = False
                child.track_running_stats = False
                
        self.intermed.rhos.reuires_grad = True
        self.intermed.locs.reuires_grad = True
                                 
    def forward(self, img, mask):
                                 
        
        x = self.intermed(img, mask)
        noisy = x.detach()
         
        x = self.model_pt2(x)                    
        x = x.view(img.size(0), -1)
        x = self.model_pt3(x)                                 

        return x, noisy
    
                                 
 






In [19]:
model = VGG_16_2()

criterion = nn.NLLLoss()
mus = torch.zeros((3,224,224))
scale = torch.ones((3,224,224))*0.001
model_syn = vgg_syn(model.convnet, model.fc ,0.0,5 ,mus, scale )
model_syn.load_state_dict(torch.load("celeba-hair-avg-cloak", map_location=device))



<All keys matched successfully>

In [20]:
mult_mask = torch.where(model_syn.intermed.scales()>4.5, torch.zeros(model_syn.intermed.scales().shape), torch.ones(model_syn.intermed.scales().shape))

In [21]:
size = model_syn.intermed.rhos.shape
ones= torch.ones(size)
mask = torch.where(model_syn.intermed.scales()>4.5, ones, torch.zeros(size))
suppressed = torch.sum(mask)
all_count = torch.sum(ones)
print(suppressed/all_count)

tensor(0.9626)


In [22]:
demographic_parity_syn(model_syn, mult_mask)

tensor(0.3461) tensor(0.4298)
tensor(-0.0837)
***********************
tensor(0.6539) tensor(0.5702)
tensor(0.0837)
0.7807890394519726


In [23]:
equal_odds_syn(model_syn, mult_mask)

tensor(2353.) tensor(2500.) tensor(1930.) tensor(2122.)
tensor(0.8202) tensor(0.8488)
tensor(-0.0286)
************
tensor(0.7746) tensor(0.7395)
tensor(0.0352)
0.7788389419470974
